In [1]:
import sys
sys.path.append('../scripts')
from utils import *

SNAKEMAKE ALL DEDUPLICATED

In [8]:
pairwise_aln_all_deduplicated = pd.read_csv('/home/tobamo/analize/project-tobamo/analysis/model/results/snakemake/testing_input.csv')
pairwise_aln_all_deduplicated['SRR'] = pairwise_aln_all_deduplicated['contig_name'].str.extract(r"_([A-Za-z0-9]+)$")
pairwise_aln_all_deduplicated.SRR.nunique()

139

In [6]:
all_deduplicated_ids = [seq.id for seq in SeqIO.parse('/home/tobamo/analize/project-tobamo/analysis/data/contigs/contigs_all_deduplicated.fasta', 'fasta')]
all_deduplicated_srrs = pd.Series(all_deduplicated_ids).str.extract(r"_([A-Za-z0-9]+)$")[0].unique()
len(all_deduplicated_ids),len(all_deduplicated_srrs)

(2549, 139)

SNAKEMAKE NON CELLULAR FILTERED

In [23]:
all_non_cellular_filtered_ids = [seq.id.replace('=','_') for seq in SeqIO.parse('/home/tobamo/analize/project-tobamo/analysis/data/contigs/contigs_non_cellular_filtered.fasta', 'fasta')]
all_non_cellular_filtered_srrs = pd.Series(all_non_cellular_filtered_ids).str.extract(r"_([A-Za-z0-9]+)$")[0].unique()
len(all_non_cellular_filtered_ids),len(all_non_cellular_filtered_srrs)

(510, 131)

SNAKEMAKE NON CELLULAR FILTERED

In [28]:
non_cellular_filteted_df = pairwise_aln_all_deduplicated[pairwise_aln_all_deduplicated.contig_name.isin(all_non_cellular_filtered_ids)]
non_cellular_filteted_df['contig_name'].nunique()

510

In [29]:
non_cellular_filteted_df.to_csv('/home/tobamo/analize/project-tobamo/analysis/model/results/snakemake/pairwise_aln_all_deduplicated_non_cellular_filtered.csv', index=False)